In [1]:
from astropy.io import fits
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd

import vice

from astropy.table import Table
import astropy as ap
import astropy.units as u

import surp

/astro/dboyea/.local/lib/python3.11/site-packages/vice-1.4.0.dev0-py3.11-linux-x86_64.egg/vice/__init__.py:141: UserWarning: Using un-released version of VICE
  warnings.warn("Using un-released version of VICE", UserWarning)


In [2]:
filename = surp._globals.DATA_DIR + "surveys/allStar-dr17-synspec_rev1.fits"
allStar = Table.read(filename, format="fits", hdu=1, memmap=True)

remove stars with
- STAR_BAD
- NO_ASPCAP_RESULT
selection effect

In [3]:
apogee_aspcapflag = 1<<23 # starbad
apogee_aspcapflag ^= 1<<31 # no_aspcap_result

In [4]:
mask = (allStar["ASPCAPFLAG"] & apogee_aspcapflag) == 0


print(sum(mask))

691591


In [5]:
apogee_target2 = 1<<17 #APOGEE_MIRCLUSTER_STAR
apogee_target2 ^= 1<<15 #APOGEE_EMISSION_STAR  emission line stars
apogee_target2 ^= 1<<13 #APOGEE_EMBEDDEDCLUSTER_STAR embedded cluster

apogee2_target3 = 1<<5 # young cluster (IN-SYNC)
apogee2_target3 ^= 1<<18 #APOGEE2_W345
apogee2_target3 ^= 1<<1 # EB planet

mask_extra = (allStar["APOGEE2_TARGET3"] & apogee2_target3) == 0
print(sum(~mask))
mask_extra &= (allStar["APOGEE_TARGET2"] & apogee_target2) == 0
print(sum(~mask))

42310
42310


In [6]:
mask &= mask_extra

In [7]:
mg_h = allStar["MG_FE"] + allStar["FE_H"]
mask &= -0.75 <= mg_h
mask &= mg_h <= 0.45
mask &= allStar["SNR"] >= 100
mask &= (mg_h < -0.5)  | (allStar["SNR"] >= 200)


print(sum(mask))

206689


In [8]:
# get rid of multdim columns
filtered = allStar[mask]
names = [name for name in filtered.colnames if len(filtered[name].shape) <= 1]
filtered = filtered[names].to_pandas()

del allStar

In [10]:
df1 = filtered

df1["MG_H"] = df1["MG_FE"] + df1["FE_H"]
df1["MG_H"] = df1["MG_FE"] + df1["FE_H"]
df1["C_H"] = df1["C_FE"] + df1["FE_H"]
df1["N_H"] = df1["N_FE"] + df1["FE_H"]
df1["C_MG"] = df1["C_H"] - df1["MG_H"]
df1["N_MG"] = df1["N_H"] - df1["MG_H"]


df1["high_alpha"] = surp.gce_math.is_high_alpha(df1["MG_FE"], df1["FE_H"])

df1.to_csv("../../data/apogee_most.csv")